# MOVED TO CASSIO EXAMPLES

# DB-backed prompts in chat sequences

### First a setup similar to the non-chat counterpart

In [ ]:
from cassandraPromptTemplate import createCassandraPromptTemplate

In [ ]:
from cqlsession import getCqlSession
astraSession = getCqlSession()

In [ ]:
# this is the prompt for a single message in the chat sequence.
# We create with the cassandra-powered data-extraction logic.
systemTemplate = """
You are a chat assistant, helping a user of age {user_age} from a city
they refer to as {city_nickname}.
"""

cassSystemPrompt = createCassandraPromptTemplate(
    session=astraSession,
    keyspace='pqdemo',
    template=systemTemplate,
    # this will work because input_variables are the actual variables passed in formatting:
    input_variables=['city', 'name'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
systemMessagePrompt = SystemMessagePromptTemplate(prompt=cassSystemPrompt)

## Putting this (DB-connected) system message into a broader chat sequence

In [ ]:
humanTemplate = "{text}"
humanMessagePrompt = HumanMessagePromptTemplate.from_template(humanTemplate)

In [ ]:
cassChatPrompt = ChatPromptTemplate.from_messages([systemMessagePrompt, humanMessagePrompt])

### Trying an all-encompassing rendering

In [ ]:
print(cassChatPrompt.format_prompt(
    city='turin',
    name='beppe',
    text='Assistant, please help me!'
).to_string())

In [ ]:
# Partialing is not implemented for ChatPromptTemplate
try:
    cassChatPartialPrompt = cassChatPrompt.partial(city='turin', name='beppe')
except NotImplementedError:
    print('"NotImplementedError" raised by partialing ChatPromptTemplate')